In [ ]:
import pyodbc
from dotenv import load_dotenv
load_dotenv()
import os

server = "SMSZUWV0077\\SASDBS"     # e.g., 'localhost\\SQLEXPRESS' or '192.168.1.100'
database = 'SAS'
username = os.getenv("USERNAME")
password = os.getenv("PWD")

conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;')
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

cursor.execute("SELECT * FROM [SAS].[dbo].[F564205A]")

for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


(11384479, 25067479, 7495328, 125136)
(11334986, 25067369, 7495349, 125136)
(11385178, 25063299, 7495392, 125136)
(11270285, 25068198, 7495394, 125136)
(11381187, 25068010, 7495421, 125136)
(11347332, 25065160, 7495485, 125136)
(11337823, 25068298, 7495501, 125136)
(11085699, 25068555, 7495627, 125136)
(11386446, 25068757, 7495633, 125136)
(11386461, 25068696, 7496074, 125139)
(11223026, 25069420, 7496087, 125139)
(11386496, 25068928, 7495650, 125136)
(11371751, 25068866, 7495652, 125136)
(11342580, 25068536, 7495658, 125136)
(11193790, 25067350, 7495661, 125136)
(11366706, 25068933, 7495670, 125136)
(11209323, 25068684, 7495678, 125136)
(11383258, 25066885, 7495713, 125136)
(11386345, 25068809, 7495726, 125136)
(673818, 25069608, 7496716, 125140)
(11026010, 25069151, 7496750, 125140)
(11149237, 25068353, 7496800, 125140)
(11244362, 25049090, 7496852, 125140)
(11190023, 25069355, 7496438, 125139)
(11358911, 25069217, 7496461, 125139)
(11090476, 25069556, 7496465, 125139)
(11129919, 250